In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Input
import tensorflow as tf
import matplotlib.pyplot as plt
from joblib import dump, load
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
import seaborn as sns

# initialize tqdm
tqdm.pandas()


In [2]:
# Function to create a confusion matrix 
def conf_matrix(y_test, pred_test):    
    
    # Creating a confusion matrix
    con_mat = confusion_matrix(y_test, pred_test)
    con_mat = pd.DataFrame(con_mat, range(2), range(2))
   
    #Ploting the confusion matrix
    plt.figure(figsize=(6,6))

    plt.title('Confusion Matrix')
    sns.set(font_scale=1.5) 
    sns.heatmap(con_mat, annot=True, annot_kws={"size": 16}, fmt='g', cmap='Blues', cbar=False)
    plt.xlabel('Predicted Values')
    plt.ylabel('Actual Values')
    

In [3]:
# Load data
df = pd.read_csv('data/995,000_row_cleaned.csv')

# Remove rows with ivalid values
label_map = {"fake": 1, "satire":1, "conspiracy": 1, "bias": 1, "rumor": 1, "junksci": 1, "hate": 1,  "clickbait": 1,   "political": 1,
             "reliable": 0} # unreliable isn't kept because it's not conclusive data 
df = df[df["type"].isin(label_map.keys())]  # Keep only rows with valid labels
print(df["type"].value_counts())
df["type"] = df["type"].map(label_map)
print(df["type"].value_counts())

# Split data 
texts = df["content"]
y = df["type"]

# Create a instances of the TF-IDF vectorizer
vectorizer = TfidfVectorizer(
    max_features=10000,  # make sure the number of features is 10000
    ngram_range=(1, 2),  # Use uni- and bi-grams
)

# Split data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(
    texts, y, 
    test_size=0.2,
    shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, 
    test_size=0.5,
    shuffle=False)

# Fit and transform the vectorizer on the training data
X_train = vectorizer.fit_transform(X_train)  
X_val = vectorizer.transform(X_val)       
X_test = vectorizer.transform(X_test)  

# Save the fitted vectorizer
dump(vectorizer, 'models/tfidf_vectorizer.joblib')

/var/folders/z1/hf1tn63x7tl1qjxy1md9xy4w0000gn/T/ipykernel_1718/4153333985.py:2: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/995,000_row_cleaned.csv')


type
reliable      218564
political     194518
bias          133232
fake          104883
conspiracy     97314
rumor          56445
clickbait      27412
junksci        14040
satire         13160
hate            8779
Name: count, dtype: int64
type
1    649783
0    218564
Name: count, dtype: int64


KeyboardInterrupt: 

In [ ]:
# Show the shape of the data
print(f"X_train.shape: {X_train.shape} \ny_train.shape: {y_train.shape} \n")
print(f"X_val.shape: {X_val.shape} \ny_val.shape: {y_val.shape}\n")
print(f"X_test.shape: {X_test.shape} \ny_test.shape: {y_test.shape}\n")


X_train.shape: (694677, 10000) 
y_train.shape: (694677,) 

X_val.shape: (86835, 10000) 
y_val.shape: (86835,)

X_test.shape: (86835, 10000) 
y_test.shape: (86835,)



In [ ]:
# Standard model
# Define the model
input_layer = Input(shape=(X_train.shape[1],), sparse=True)
x1 = layers.Dense(1000, activation="relu")(input_layer) # 1st hidden layer with 1000 neurons
x1 = layers.Dropout(0.4)(x1)                            # Dropout layer to prevent overfitting 
x2 = layers.Dense(500, activation="relu")(x1)           # 2nd hidden layer with 500 neurons
x2 = layers.Dropout(0.2)(x2)                            # Dropout layer to prevent overfitting
x3 = layers.Dense(100, activation="relu")(x2)           # 3rd hidden layer with 100 neurons
x4 = layers.Dense(25, activation="relu")(x3)            # 4th hidden layer with 25 neurons
output = layers.Dense(1, activation="sigmoid")(x4)      # Output layer with 1 neuron and sigmoid to force binary

# Create the model
NN = tf.keras.models.Model(inputs=input_layer, outputs=output)

#Initialize the NN
NN.compile(
    optimizer = "adam",
    loss = "binary_crossentropy",
    metrics=[
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall(),
        tf.keras.metrics.F1Score()],
    weighted_metrics=["f1_score"],  # Weight the F1 metric higher
)

# Define the callbacks for early stopping
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_f1_score',     # Use F1 for early stopping
        mode='max',                 # We want to "maximize" the F1 score  
        patience=4,                 # If theres no improvements after 4 epochs, stop
        restore_best_weights=True
    )
]

# Define the weights for the classes
weight_for_0 = len(y_train) / (2.0 * (y_train == 0).sum())
weight_for_1 = len(y_train) / (2.0 * (y_train == 1).sum())
class_weights = {0: weight_for_0, 1: weight_for_1}

#Train the NN
history = NN.fit(
    X_train,y_train,                    # Trainin data as input and expected output
    validation_data = (X_val,y_val),    # Data for validation
    epochs = 10,                        # Number of iterations over the entire dataset
    batch_size = 1024,                  # Number of samples per gradient update
    callbacks=callbacks,                # Use the callback
    class_weight=class_weights          # Use the class weights
)

Epoch 1/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 52s 75ms/step - f1_score: 0.8627 - loss: 0.2593 - precision_1: 0.9552 - recall_1: 0.8980 - weighted_f1_score: 0.6744 - val_f1_score: 0.8730 - val_loss: 0.1828 - val_precision_1: 0.9619 - val_recall_1: 0.9404 - val_weighted_f1_score: 0.8730
Epoch 2/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 51s 76ms/step - f1_score: 0.8984 - loss: 0.1484 - precision_1: 0.9762 - recall_1: 0.9490 - weighted_f1_score: 0.7445 - val_f1_score: 0.8887 - val_loss: 0.1724 - val_precision_1: 0.9631 - val_recall_1: 0.9407 - val_weighted_f1_score: 0.8887
Epoch 3/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 52s 77ms/step - f1_score: 0.9098 - loss: 0.0818 - precision_1: 0.9879 - recall_1: 0.9735 - weighted_f1_score: 0.7688 - val_f1_score: 0.9010 - val_loss: 0.1924 - val_precision_1: 0.9593 - val_recall_1: 0.9512 - val_weighted_f1_score: 0.9010
Epoch 4/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 52s 76ms/step - f1_score: 0.9172 - loss: 0.0525 - precision_1: 0.9912 - recall_1: 0.9849 - weighted_f1_score: 0.7851 - v

In [ ]:
# Save the model
output_path = "models/nn_classifier12_bigram.keras"
NN.save(output_path) 

In [5]:
# Load the model
input_path = "models/nn_classifier12_bigram.keras"
NN = tf.keras.models.load_model(input_path)

In [ ]:
# Evaluate on test set
test_results = NN.evaluate(X_test, y_test, verbose=1)
print(f"* Test Loss: {test_results[0]:.4f}")
print(f"* Test Accuracy: {test_results[1]:.4f}")
print(f"* Test Precision: {test_results[2]:.4f}")
print(f"* Test Recall: {test_results[3]:.4f}")
print(f"* Test AUC: {test_results[4]:.4f}")
print(f"* Test F1 Score: {test_results[5]:.4f}")